## Iniciando Modelo de Entrenamiento
Para esta parte, el articulo menciona el uso de modelo orientado a Random Forest con Regresion, actualmente ya se tiene el dataset final con mas de 34000 registros y 72 caracteristicas.
Se utilizara TensorFlow corriendo con Cuda para el entrenamiento y testeo

In [38]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [39]:
print(tf.__version__)

2.18.0


In [40]:
import torch

print(torch.cuda.is_available())

True


### Importamos el dataset

In [41]:
import pandas as pd
import numpy as np

In [42]:
dataset_df = pd.read_csv("main_dataset_without_glucose.csv")
dataset_df.head()

,datetime,nivel,gender,hba1c,id,acc_mean,acc_median,acc_max,acc_min,acc_std,...,food2h_sugar_sum,food2h_sugar_mean,food2h_protein_sum,food2h_protein_mean,food2h_cantidad_comidas_,food2h_media_comidas_,food8h_cantidad_comidas_,food8h_media_comidas_,food24h_cantidad_comidas_,food24h_media_comidas_
0,2020-02-13 17:20:00,PersNorm,FEMALE,5.5,1.0,64.328161,63.976558,85.545310,47.031904,4.610962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-02-13 17:25:00,PersNorm,FEMALE,5.5,1.0,64.632465,63.921827,90.365923,40.853396,7.454712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-02-13 17:30:00,PersNorm,FEMALE,5.5,1.0,64.531001,64.148266,88.345911,41.060930,5.465436,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-13 17:35:00,PersNorm,FEMALE,5.5,1.0,63.708273,63.095166,149.298359,15.132746,6.181412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-13 17:40:00,PersNorm,FEMALE,5.5,1.0,63.475463,63.063460,157.711762,14.866069,5.757218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
label = 'nivel'
classes = dataset_df['nivel'].unique().tolist()
print(classes)

['PersNorm', 'PersHigh', 'PersLow']


In [44]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [45]:
train_ds_pd, test_ds_pd = split_dataset(dataset_df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

24338 examples in training, 10254 examples for testing.


In [46]:
import ydf
model = ydf.GradientBoostedTreesLearner(label=label).train(train_ds_pd)

Train model on 24338 examples


max_vocab_count = -1 for column nivel, the dictionary will not be pruned by size.
"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 24338
Number of columns: 71

Number of columns by type:
	NUMERICAL: 68 (95.7746%)
	CATEGORICAL: 3 (4.22535%)

Columns:

NUMERICAL: 68 (95.7746%)
	3: "hba1c" NUMERICAL mean:5.75416 min:5.3 max:6.4 sd:0.271645 dtype:DTYPE_FLOAT64
	4: "id" NUMERICAL mean:7.7628 min:1 max:15 sd:4.1981 dtype:DTYPE_FLOAT64
	5: "acc_mean" NUMERICAL mean:50.4745 min:0 max:109.018 sd:26.6225 dtype:DTYPE_FLOAT64
	6: "acc_median" NUMERICAL mean:50.1336 min:0 max:109.895 sd:26.4305 dtype:DTYPE_FLOAT64
	7: "acc_max" NUMERICAL mean:97.9295 min:0 max:221.702 sd:66.8422 dtype:DTYPE_FLOAT64
	8: "acc_min" NUMERICAL mean:26.7861 min:0 max:66.0076 sd:22.4841 dtype:DTYPE_FLOAT64
	9: "acc_std" NUMERICAL mean:3.4633

Model trained in 0:00:14.947679


Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.269746
Truncates the model to 486 tree(s) i.e. 162  iteration(s).
Final model num-trees:162 valid-loss:0.269746 valid-accuracy:0.886023


In [47]:
model.describe()

In [48]:
model.predict(test_ds_pd)

Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Engine "GradientBoostedTreesGeneric" built


array([[6.7225859e-02, 9.3277228e-01, 1.8186445e-06],
       [2.1084953e-02, 9.7891319e-01, 1.7415944e-06],
       [7.7688836e-02, 9.2231017e-01, 9.1943292e-07],
       ...,
       [2.1696633e-01, 7.8303146e-01, 2.3421112e-06],
       [2.1696633e-01, 7.8303146e-01, 2.3421112e-06],
       [2.1696633e-01, 7.8303146e-01, 2.3421112e-06]], dtype=float32)

In [51]:
# Analyse a model (e.g. partial dependence plot, variable importance)
model.analyze(test_ds_pd)

# Benchmark the inference speed of a model
model.benchmark(test_ds_pd)

Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Engine "GradientBoostedTreesGeneric" built
Run the model with the fast engine wrapper
List plotting attribute combinations
Found 70 combination(s)
Initiate PDP accumulator
Compute partial dependence plot for 70 set of features and 2568 model call(s) per example.
1 examples scanned.
Maximum duration reached. Interrupting analysis early.
List plotting attribute combinations
Found 70 combination(s)
Initiate CEP accumulator
Compute conditional expectation plot for 70 set of features and 2568 model call(s) per example.
1 examples scanned.
Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Engine "GradientBoostedTreesGeneric" built
Running 71 features on 12 threads with 1 rounds
Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Model loaded with 486 root(s), 25120 node(s), and 56 input feature(s).
Model loaded w

Single-thread inference time per example: 15.844 us (microseconds)
Details: 215334 predictions in 0.000 seconds

Multi-thread inference time per example: 2.115 us (microseconds)
Details: 1353528 predictions in 0.000 seconds using 12 threads

* Measured with the C++ serving API. See model.to_cpp().

In [49]:
evaluation = model.evaluate(test_ds_pd)

# Query individual evaluation metrics
print(f"test accuracy: {evaluation.accuracy}")

# Show the full evaluation report
print("Full evaluation report:")
evaluation

test accuracy: 0.8871659840062415
Full evaluation report:


Label \ Pred,PersHigh,PersNorm,PersLow
PersHigh,5383,550,0
PersNorm,466,3581,105
PersLow,0,36,133
